<a href="https://colab.research.google.com/github/saugatbh/RainfallDataAnalysis/blob/main/RainfallCNN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Read 114 images**

In [ ]:
import cv2
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten
from keras.utils import np_utils
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot


path='/content/drive/MyDrive/RainfallDataAnalysis-main/ModifiedFiguresAnnualAvg'
images=[]
for filename in os.listdir(path):
  img = cv2.imread(os.path.join(path,filename))
  if img is not None:
    images.append(img)
images=np.array(images)
images = images.astype('float32')
images /= 255

**Define CNN**

In [ ]:
def define_model():
  model = Sequential()
  model.add(Conv2D(50,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',input_shape=(656,875,3)))
  model.add(Conv2D(75,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(125,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  
  model.add(Dense(500,activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(250,activation='relu'))
  model.add(Dropout(0.3))
  
  model.add(Dense(10,activation='softmax'))
  
  model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
  
  return model

**Split training and testing** <br>
Last 14 images for testing

In [ ]:
def train_test_split(data,ntest):
  print("splitting data")
  return data[:ntest], data[-ntest:]

**Series to Supervised**

In [ ]:
def split_data(images, nstep=15):
	X, y = list(), list()
	for i in range(len(images)):
		# find the end of this pattern
		end_ix = i + nstep
		# check if we are beyond the sequence
		if end_ix > len(images)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = images[i:end_ix], images[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)






def series_to_supervised(data, n_in, n_out=1):
      
    df = pd.DataFrame(data)
    print(df.describe)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    print('converted data is ----------')
    print(agg.values)
    agg = np.array(agg).reshape(n_in,656,875,3)
    return agg

**Model Fit**

In [ ]:
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# fit a model
def model_fit(train):
    print("here in model fit")
    # prepare data
    data = series_to_supervised(train, 5)
    #train_x, train_y = split_data(train,nstep=15)
    train_x, train_y = data[:, :-1], data[:, -1]
    print("done to supervised")
    print(train_x.shape,train_y.shape)
    # define model
    model = define_model()
    # fit
    model.fit(train_x, train_y, epochs=10, batch_size=128, verbose=0)
    return model

# forecast with a pre-fit model
def model_predict(model, history):
    # prepare data
    print("here in predict")
    x_input = array(history[-5:]).reshape(1, 5)
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return yhat[0]

**Walk forward validation**

In [ ]:
def walk_forward_validation(data,n_test):
  print("here in walk")
  predictions = list()
  train, test = train_test_split(data, n_test)
  print(type(train))
  print(train.shape)
  print(test.shape)
  model = model_fit(train)
  history = [x for x in train]
  for i in range(len(test)):
    yhat = model_predict(model, history)
    predictions.append(yhat)
    history.append(test[i])
  error = measure_rmse(test, predictions)
  print(' > %.3f' % error)
  return error

**Evaluate**

In [ ]:
def repeat_evaluate(data, n_test, n_repeats=3):
  
  print('here!')
  print(data.shape)
  data = data.reshape(data.shape[0],(656*875*3))
  scores = [walk_forward_validation(data, n_test) for _ in range(n_repeats)]
  scores_m, score_std = np.mean(scores), np.std(scores)
  print('%s: %.3f RMSE (+/- %.3f)' % (scores_m, score_std))
  pyplot.boxplot(scores)
  pyplot.show()

**Main Function call**

In [ ]:
n_test=14
repeat_evaluate(images,n_test)